In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import os
from sklearn.metrics import mean_squared_error
import random
import math
import time
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from random import choice
import pandas as pd
from sklearn.linear_model import Lasso

In [2]:
# Write a block generator to obtain different initial data sets

#  n is the number of parts into which it is divided, i is the serial number of the block/batch
def getChunks(data,n,i):
    return data[ int(len(data)*(i)/n)+1 : (int(len(data)*(i+1)/n)-1)]
def getPourCentage(data,beginCentage,endCentage):
    return data[ (int(len(data)*beginCentage)): (int(len(data)*endCentage))]

#Function description: return initial data, return all training data, determine if a next batch exists, return current batch.
class feeder_Ini_Train_Batch():
    def __init__(self,X,Y,beginCentage,endCentage,batch_size):
        self.X = X
        self.Y = Y
        #elf.X_train = getPourCentage(X,0,iniCentage)
        #elf.Y_train = getPourCentage(Y,0,iniCentage)
        self.X_test = getPourCentage(X,beginCentage,endCentage)
        self.Y_test = getPourCentage(Y,beginCentage,endCentage)
        self.batch_size = batch_size
        self.t = 1 # ‘t’ is indexing and time. 
    def getIni_X_Y(self,iniCentage):
        return getPourCentage(self.X,0,iniCentage),getPourCentage(self.Y,0,iniCentage)
    def getTrain_X_Y(self):
        return self.X_test,self.Y_test
    def hasThisBatch(self):
        if (self.t)*self.batch_size < len (self.X_test):
            return True
        else:
            return False
    def hasThisBatch_and_nextBath(self):
        if (self.t+1)*(self.batch_size) < len (self.X_test):
            return True
        else:
            return False
    def getThisBatch(self):
        if self.hasThisBatch() == True:
            actul_Batch_X = self.X_test[(self.t-1)*self.batch_size:(self.t)*self.batch_size]
            actul_Batch_Y = self.Y_test[(self.t-1)*self.batch_size:(self.t)*self.batch_size]
            return actul_Batch_X,actul_Batch_Y
        else:
            print('err index out')
    def getNextBatch_getThisBatch(self):
        if self.hasThisBatch() == True:
            next_Batch_X = self.X_test[(self.t)*self.batch_size:(self.t+1)*self.batch_size]
            next_Batch_Y = self.Y_test[(self.t)*self.batch_size:(self.t+1)*self.batch_size]
            return next_Batch_X,next_Batch_Y
        else:
            print('err index out')
    def goNext(self):
        self.t +=1



In [3]:
def getMeanSuperPourCentage(data,PourCentage): # Calculate the average value of （the top percentage of） the data. 
#     print(np.percentile(data, (PourCentage)))
    return np.mean([x for x in data if x >= np.percentile(data, (PourCentage))])
def getMeanSuperPourCentage_Martix_ParLine(data,PourCentage):
    return np.array([getMeanSuperPourCentage(x,PourCentage) for x in data ])

# 0 submodel 1 numbering time 2 weights 3 class name 4 common name 5 varepsilon weighting error 6 Omega time weighting 7 err common RMSE error 8 model address

def RandonSelectionModle(index,x,y , optitionInnErrOrCroissErr,numFlod = 3, indique_sousModle = 'Random_LS_LI_R_T'):  

    if indique_sousModle == 'Random_LI_R_T_KNN':
        indique_sousModle = choice(['Lasso','R_Forest','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_LI_R_T_KNN':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_LI_R_T_SVR':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','Tree','SVR'])
#     elif indique_sousModle == 'Random_LS_LI_R_T':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','Tree'])
#     elif indique_sousModle == 'Random_LS_LI_R_KNN':
#         indique_sousModle = choice(['LSTM','Lasso','R_Forest','KNN'])
#     elif indique_sousModle == 'Random_LS_LI_T_KNN':
#         indique_sousModle = choice(['LSTM','Lasso','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_R_T_KNN':
#         indique_sousModle = choice(['LSTM','R_Forest','Tree','KNN'])
#     elif indique_sousModle == 'Random_LS_R_T':
#         indique_sousModle = choice(['LSTM','R_Forest','Tree'])
#     elif indique_sousModle == 'Random_LS_LI_T':
#         indique_sousModle = choice(['LSTM','Lasso','Tree'])
        
        print('Randomly selected indique_sousModle ',indique_sousModle)
    weight = 1 # weight of  this base model 
    varepsilon = 1 # Results of the loss functions
    omega = 1 #  Weight of time
    RMSE = 1 # Common RMSE error
    x_rnn = np.reshape(x, (x.shape[0], 1,x.shape[1]))
    kf = KFold(n_splits=numFlod)
    yhat_all_flod = []
    

    
#     elif indique_sousModle == 'LSTM' :
#         modele = Sequential()
#         modele.add(LSTM(500, input_shape =(1, x_rnn.shape[2]) , activation='relu'))
#         modele.add(Dense(500, activation='relu'))
#         modele.add(Dense(y.shape[1] , activation='sigmoid'))
#         # Compile model
#         modele.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#         modele.fit(x_rnn, y, epochs=6, batch_size=2000,  verbose=2)
#         if optitionInnErrOrCroissErr == 'CroissErr':
#             for train, test in kf.split(x_rnn):
#                 X_train_numFlod = np.array( [x_rnn[i] for i in train])
#                 Y_train_numFlod = np.array([y[i] for i in train])
#                 X_test_numFlod = np.array([x_rnn[i] for i in test])
#                 Y_test_numFlod = np.array([y[i] for i in test])
#                 modele_numFlod = Sequential()
#                 modele_numFlod.add(LSTM(500, input_shape =(1, x_rnn.shape[2]) , activation='relu'))
#                 modele_numFlod.add(Dense(500, activation='relu'))
#                 modele_numFlod.add(Dense(y.shape[1] , activation='sigmoid'))
#                 modele_numFlod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#                 modele_numFlod.fit(X_train_numFlod, Y_train_numFlod, epochs=4, batch_size=1000,  verbose=2)
#                 yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
#                 yhat_all_flod += yhat__numFlod.tolist()
#         return [modele,index,weight,modele.__class__,'LSTM',varepsilon,omega,RMSE,id(modele)] , yhat_all_flod
    
    
    
#     elif indique_sousModle == 'SVR' :
#         modele = MSVR_F()
#         modele.fit(x,y)
#         if optitionInnErrOrCroissErr == 'CroissErr':
#             for train, test in kf.split(x):
#                 X_train_numFlod = np.array( [x[i] for i in train])
#                 Y_train_numFlod = np.array([y[i] for i in train])
#                 X_test_numFlod = np.array([x[i] for i in test])
#                 Y_test_numFlod = np.array([y[i] for i in test])
#                 modele_numFlod = MSVR_F()
#                 modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
#                 yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
#                 yhat_all_flod += yhat__numFlod.tolist()        
#         return [modele,index,weight,modele.__class__,'SVR',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
    

    if indique_sousModle == 'KNN':
        modele = KNeighborsRegressor(n_neighbors=15)
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = KNeighborsRegressor(n_neighbors=15)
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()        
        return [modele,index,weight,modele.__class__,'KNeighborsRegressor',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
    
    elif indique_sousModle == 'R_Forest':
        modele = RandomForestRegressor(n_estimators=10, )
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = RandomForestRegressor(n_estimators=10,)
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()
        return [modele,index,weight,modele.__class__,'RandomForestRegressor',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
            
    elif indique_sousModle == 'Lasso':
        modele = Lasso(alpha=0.1)
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = Lasso(alpha=0.1)
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()        
        return [modele,index,weight,modele.__class__,'LassoRegression',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
        
    elif indique_sousModle == 'Tree':
        modele = DecisionTreeRegressor()
        modele.fit(x,y)
        if optitionInnErrOrCroissErr == 'CroissErr':
            for train, test in kf.split(x):
                X_train_numFlod = np.array( [x[i] for i in train])
                Y_train_numFlod = np.array([y[i] for i in train])
                X_test_numFlod = np.array([x[i] for i in test])
                Y_test_numFlod = np.array([y[i] for i in test])
                modele_numFlod = DecisionTreeRegressor()
                modele_numFlod.fit(X_train_numFlod,Y_train_numFlod)
                yhat__numFlod = modele_numFlod.predict(X_test_numFlod)
                yhat_all_flod += yhat__numFlod.tolist()        
        return [modele,index,weight,modele.__class__,'DecisionTreeRegressor',varepsilon,omega,RMSE,id(modele)],yhat_all_flod
        
def makePredictionModele(modele,x):
    if type(modele) in [type(RandomForestRegressor()), 
                          type(KNeighborsRegressor()),
                          type(Lasso()),
                          type(DecisionTreeRegressor())] :

        return  modele.predict(x)
    else:
        print('Error reported, not in the list')

def beta_fonction(k,t,a,b):  
    return 1/(1+ math.exp(-a*(t-k-b))) 

def vote_mean(dic_expert, actul_Batch_X, actul_Batch_Y, afficher_detail = False):
    H_res = np.array( [[0.0]* len(actul_Batch_Y[0])] *len(actul_Batch_Y))
    print(H_res)
    sumWeigt = 0
    
    list_RMSE_SousModele = []

    for key,value in dic_expert.items():
        yhat_sousM = makePredictionModele(value[0],actul_Batch_X)
        
        if np.array(yhat_sousM).ndim == 1:
            print('type([[z] for z in yhat_sousM])  ',type([[z] for z in yhat_sousM]))
            yhat_sousModel = np.array([[z] for z in yhat_sousM])
        else:
            yhat_sousModel = yhat_sousM
            print(type(yhat_sousModel))
        print('Output of each sub-model',yhat_sousModel)
        H_res =H_res + yhat_sousModel*value[2]
        sumWeigt += value[2] #  2 2 represents the vote weights of the sub model 
        list_RMSE_SousModele.append(aRMSE( actul_Batch_Y ,yhat_sousModel))
    yhat_H = np.array(H_res)/sumWeigt # Obtaining the predicted results 
    
#     print('Output after the end of voting', yhat_H)
#     print('Original value of Y', actul_Batch_Y)
    if afficher_detail == True:
        print( 'list_RMSE_SousModele ', list_RMSE_SousModele, 'mean of list_RMSE_SousModele', np.mean(list_RMSE_SousModele) )
        print(' RMSE resultat vote', aRMSE( actul_Batch_Y ,yhat_H))
    return yhat_H


def vote_OnlyMaxERRWeight(dic_expert, actul_Batch_X, actul_Batch_Y):
    the_key = 0
    maxWeight = 0
    for key,value in dic_expert.items():
        if dic_expert[key][5] > maxWeight:
            maxWeight = dic_expert[key][5]
            the_key = key
    yhat_H = makePredictionModele(dic_expert[the_key][0],actul_Batch_X)
    err_H =  aRMSE(actul_Batch_Y,yhat_H)# err as RMSE 
    print('Sub-model selected to vote, no. ', the_key, ' with its RMSE', err_H)
    return yhat_H

def vote_OnlyMaxESpWeight(dic_expert, actul_Batch_X, actul_Batch_Y):
    the_key = 0
    maxWeight = 0
    for key,value in dic_expert.items():
        if dic_expert[key][2] > maxWeight:
            maxWeight = dic_expert[key][2]
            the_key = key
    yhat_H = makePredictionModele(dic_expert[the_key][0],actul_Batch_X)
    err_H =  aRMSE(actul_Batch_Y,yhat_H)# err as RMSE 
    print('Sub-model selected to vote, no. ', the_key, ' with its RMSE', err_H)
    return yhat_H

def updatingALLSousModele(dic_expert, x, y):
    print (' Update sub-model program start' )
    x_rnn = np.reshape(x, (x.shape[0], 1,x.shape[1]))
    for key,value in dic_expert.items():
        if type(value[0]) in [type(Sequential())]:
            value[0].fit(x_rnn, actul_Batch_Y)

            
def get_filename(path,filetype):  # Enter the path, file type e.g. '.csv'
    name = []
    for root,dirs,files in os.walk(path):
        for i in files:
            if os.path.splitext(i)[1]==filetype:
                name.append(i)    
    return name

def aRMSE(y_true,y_pred):
    return np.mean(mean_squared_error(y_true, y_pred, squared=False, multioutput='raw_values'))

In [4]:


path_X_Y_data = 'simulationDonnes/'
Y = np.load(path_X_Y_data + 'Y_CU_BEMS_19_normale_nettoy.npy' ,allow_pickle=True)
X = np.load(path_X_Y_data + 'X_CU_BEMS_19_normale_nettoybackTimePoint3PCA10.npy' ,allow_pickle=True)


while(True):
    dic_expert = {} # Maintain a dictionary of submodels
    timeList, errList, innErrList, testErrNextBatchList,nFlodErr= [],[],[],[],[] #Time list, accuracy list etc.
    Resulta_list, Ture_list = [],[] # Recorded information: predicted results and true values
    ini_pourCentage = 0.01 # What percentage of data is used to initialize 
    testStar,testEnd = 0.2,0.99
    HowManyDaysForBatch =14
    batch_size = 1400*HowManyDaysForBatch# Block size
    a,b = 0.35,1
    time_Ini = 0 # The time required for initialization is recorded here
    varepsilon_extreme_bord = 1.2 # Coefficient to check if the sub-model is working correctly 
    maxNumSousModle = 10 # Maximum number of sub-models
    testBatchNum = 100 # Control the amount of batchs during the experiment
    # optionPCA, PCA_size = True, 25
    optionPCA, PCA_size = False, 25 # if the test dataset has been processed by PCA
    # optitionInnErrOrCroissErr = 'InnErr'
    optitionInnErrOrCroissErr = 'CroissErr'
    optitionAddSousModele = True
    optionVote ='Mean'
    # optionVote ='OnlyMaxERRWeight'
    # optionVote = 'vote_OnlyMaxESpWeight'
    indicateur_sousModel = 'Random_LI_R_T_KNN' #['Random','LSTM','Lasso','KNN','R_Forest'] 
    cStepAugment = 1.4
    updateOrNon = False


    if optionPCA == True:
        pca = PCA(n_components=PCA_size)
        principalComponents = pca.fit_transform(X)
        pca.explained_variance_ratio_
        importance = pca.explained_variance_ratio_
        plt.scatter(range(1,PCA_size+1),importance)
        plt.plot(range(1,PCA_size+1),importance)
        plt.title('Scree Plot')
        plt.xlabel('Factors')
        plt.ylabel('Eigenvalue')
        plt.grid()
        plt.show()
        X = pca.fit_transform(X)
    
    myFeeder = feeder_Ini_Train_Batch(X,Y,testStar,testEnd,batch_size) # X,Y beginCentage,endCentage,batch_size 
    X_train,Y_train = myFeeder.getIni_X_Y(ini_pourCentage) 
    X_test,Y_test = myFeeder.getTrain_X_Y()

    iniFlage = True

    while ((testBatchNum > myFeeder.t)  and myFeeder.hasThisBatch()):

        # Create an initialized sub-model, index 0
        start_Ini = time.process_time()# Recording start time
        if iniFlage == True:
            Delta = [1/ myFeeder.batch_size]* myFeeder.batch_size 
            dic_expert[0], err_numFlod= RandonSelectionModle(0,X_train, Y_train,optitionInnErrOrCroissErr, indique_sousModle = indicateur_sousModel)
            iniFlage = False
        time_Ini = round(time.process_time()-start_Ini,5) # The time required for initialization is recorded here
        start = time.process_time()# Recording start time 
        
        #----------------VOTE------------------------------------------------------
        actul_Batch_X, actul_Batch_Y  =  myFeeder.getThisBatch()
        # Calculer la performance de Ht-1 à t 
        if optionVote == 'Mean':
            yhat_H = vote_mean(dic_expert,actul_Batch_X, actul_Batch_Y,afficher_detail=True)
        elif optionVote == 'Max':
            yhat_H = vote_OnlyMaxWeight(dic_expert,actul_Batch_X, actul_Batch_Y)

        print('vote is over')
        #----------------------------------------------------------------------

        #----------------Confirm instance weights and record current cluster performance------------------------------------------------------
        ERR_absolu_H = np.abs(actul_Batch_Y - yhat_H) #ERR_absolu (martix)
        Resulta_list.append(yhat_H)
        Ture_list.append(actul_Batch_Y)
        err_H = aRMSE(actul_Batch_Y,yhat_H)# err as RMSE 
        meanERR_absolu_H = np.mean(ERR_absolu_H, axis=1)

        varepsilon_H = np.average(meanERR_absolu_H)
        errList.append(err_H) 
        print('Instance weights, recording current group performance, over')
        #----------------Confirm instance weights and record current cluster performance------------------------------------------------------

        #----------------Alarm in case of abnormality------------------------------------------------------
        if err_H >10:
            print('time: ',myFeeder.t,'Une erreur majeure s est produite：',err_H) 
        #----------------Alarm in case of abnormality------------------------------------------------------


        #------------------Add a new sub-model and confirm its weights (from cross-validation)-----------------------------
        bord = 1 
        while(optitionAddSousModele):
            print('Start adding new sub-models')
            dic_expert[myFeeder.t], yhat_new_numFlod = RandonSelectionModle(myFeeder.t, actul_Batch_X, actul_Batch_Y, optitionInnErrOrCroissErr,indique_sousModle = indicateur_sousModel)

        
            if optitionInnErrOrCroissErr == 'CroissErr':
                ERR_absolu_new_numFlod = np.abs(actul_Batch_Y - yhat_new_numFlod) 
                meanERR_absolu_new_numFlod = np.mean(ERR_absolu_new_numFlod, axis=1)
                err_new_numFlod = aRMSE(actul_Batch_Y,yhat_new_numFlod) 
                varepsilon_new = np.average(meanERR_absolu_new_numFlod,weights=Delta)
                if varepsilon_new < varepsilon_H*bord:
                    nFlodErr.append(err_new_numFlod)
                    break
                else:
                    bord = bord*cStepAugment
                
            elif optitionInnErrOrCroissErr == 'InnErr':

                yhat_new_inner = makePredictionModele(dic_expert[myFeeder.t][0], actul_Batch_X)
                ERR_absolu_new_inner = np.abs(actul_Batch_Y - yhat_new_inner) 
                meanERR_absolu_new_inner = np.mean(ERR_absolu_new_inner, axis=1)
                err_new_inner = aRMSE(actul_Batch_Y,yhat_new_inner) 
                varepsilon_new = np.average(meanERR_absolu_new_inner,weights=Delta)
                break
        #------------------Add a new sub-model and confirm its weights (from cross-validation)-----------------------------
        
        if optitionAddSousModele:
            #------------------Recording training errors for new sub-models-----------------------------
            yhat_new_inner = makePredictionModele(dic_expert[myFeeder.t][0], actul_Batch_X)
            err_new_inner = aRMSE(actul_Batch_Y,yhat_new_inner)         
            innErrList.append(err_new_inner)
            #------------------Recording training errors for new sub-models-----------------------------

            #------------------Test error of the model generated by the previous block in the next block-----------------------------
            if myFeeder.hasThisBatch_and_nextBath():
                next_batch_test_X,  next_batch_test_Y =  myFeeder.getNextBatch_getThisBatch()
                yhat_next_batch_test = makePredictionModele(dic_expert[myFeeder.t][0], next_batch_test_X)
                err_test_NextBatch = aRMSE(next_batch_test_Y, yhat_next_batch_test)         
                testErrNextBatchList.append(err_test_NextBatch)
            #------------------Test error of the model generated by the previous block in the next block-----------------------------


        #------------------Calculate the performance of the old sub-model based on the weighted strengths in the current block-----------
        Varepsilon_list = [] #Collection of the list of separate weighings of Varepsilon to facilitate the search for max-min values.
        for key,value in dic_expert.items():
            yhat_oldModel = makePredictionModele(value[0], actul_Batch_X)
            ERR_absolu_oldModel = np.abs(actul_Batch_Y - yhat_oldModel)
            mean_ERR_absolu_oldModel = np.mean(ERR_absolu_oldModel, axis=1)
            err_oldModel = aRMSE(actul_Batch_Y,yhat_oldModel) # RMSE
            varepsilon_oldModel = np.average(mean_ERR_absolu_oldModel,weights=Delta)
    #         if varepsilon_oldModel > varepsilon_H*varepsilon_extreme_bord: 
    #             value[5] = varepsilon_H*varepsilon_extreme_bord
    #         else:
    #             value[5] = varepsilon_oldModel
            value[5] = varepsilon_oldModel
            Varepsilon_list.append(value[5])
            value[7] = err_oldModel # enregrister RMSE
    #         for key,value in dic_expert.items():   #normalisation 
    #             value[5] = (value[5] - min(Varepsilon_list))/(max(Varepsilon_list) - min(Varepsilon_list))
    #     for key,value in dic_expert.items(): 
    #         print('key ', key, ' value[5] ', value[5] )
        #------------------ calculates the performance of the old sub-model based on the weighted strengths in the current block (common error #7) and updates the weights (#5 based on weighting errors) -----------


        #------------------Calculating time weights ----------
        for key,value in dic_expert.items(): 
            numerator = beta_fonction(value[1],myFeeder.t,a,b)
            denominator = 0
            for j in range(0, myFeeder.t - value[1] +1): # 
                denominator +=  beta_fonction(myFeeder.t - j,myFeeder.t,a,b)
#             print('denominator  ' , denominator)
#             if  denominator == 0:
#                 print(' myFeeder.t  ', myFeeder.t,' value[1]', value[1] )
#                 for j in range(0, myFeeder.t - value[1] +1): # 
#                     print('j :', j , ' beta_fonction(myFeeder.t - j,myFeeder.t,a,b)', beta_fonction(myFeeder.t - j,myFeeder.t,a,b))
            value[6] =  numerator/denominator
        for key,value in dic_expert.items(): 
            print('key ', key, ' value[6] ', value[6] )
        #------------------Calculating time weights ----------


        #------------------Calculating sub-model weights-----------
        # Calculer les poids des sous-modèles
        weight_list = []
        for key,value in dic_expert.items(): 
            denominator = 0 
            for j in range(0, myFeeder.t - value[1] +1): #  from 0 to  t-k
                if ((myFeeder.t-j) in dic_expert.keys()):
                    denominator += dic_expert[myFeeder.t-j][6]*dic_expert[myFeeder.t-j][5]+0.1  # Omega weight * Error of current block being weighted    
            value[2] = math.log(1/denominator)  
            weight_list.append(math.log(1/denominator)) 
        for key,value in dic_expert.items(): 
            value[2] = (value[2] - min(weight_list)+0.1)/(max(weight_list) - min(weight_list)+0.1)
        for key,value in dic_expert.items(): 
            print('key ', key, 'value[2] ',value[2] )
        #------------------Calculating sub-model weights-----------


        #------------------Removing the worst model -----------
        if len(dic_expert) > maxNumSousModle:
            maxErr = 0
            del_key = 0
            for key,value in dic_expert.items(): 
                if dic_expert[key][7] > maxErr:
                    maxErr = dic_expert[key][7]
                    del_key = key
            dic_expert.pop(del_key)
        #------------------Removing the worst model -----------
        
        
        #------------------Whether the sub-model LSTM is updated----------- # 《This feature was deprecated》
        if updateOrNon:
            updatingALLSousModele(dic_expert,actul_Batch_X,actul_Batch_Y) 
        #------------------Whether the sub-model LSTM is updated-----------


        # Recording time
        timeList.append(round(time.process_time()-start,3))

        print('Average errors so far np.mean(errList)', np.mean(errList))    #Show average of err
        print('Current Error err_H', err_H) #Show err in Current

        #Go to the next batch
        print('myFeeder.t ', myFeeder.t)
        myFeeder.goNext()

        # print(errList)

    #------------Store partial results-----------------
    patch = "resulta/"
    nameFileSave = "CUBEM_resulta"
    # nameFileSave = "UKDALE_resulta"
    res_oneTour = {}
    res_oneTour['Resulta_list'] =  Resulta_list
    res_oneTour['Ture_list'] =  Ture_list
    np.save(patch + nameFileSave + '.npy', res_oneTour , allow_pickle=True, fix_imports=True)

    break;


Randomly selected indique_sousModle  R_Forest
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
Output of each sub-model [[0.         0.00826631 0.01272912 ... 0.01726049 0.         0.        ]
 [0.         0.00705987 0.01283096 ... 0.01694378 0.         0.        ]
 [0.         0.00174263 0.01639511 ... 0.01797308 0.         0.        ]
 ...
 [0.46385984 0.36179625 0.31670061 ... 0.02889945 0.11295861 0.44907162]
 [0.45765915 0.35924933 0.31863544 ... 0.02842439 0.11223568 0.45225464]
 [0.41753807 0.37645219 0.3395112  ... 0.03230404 0.11852521 0.47639257]]
list_RMSE_SousModele  [0.0978164999965197] mean of list_RMSE_SousModele 0.0978164999965197
 RMSE resultat vote 0.0978164999965197
vote is over
Instance weights, recording current group performance, over
Start adding new sub-models
Randomly selected indique_sousModle  KNN
key  0  value[6]  0.5474158342212

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


Start adding new sub-models
Randomly selected indique_sousModle  R_Forest
key  0  value[6]  0.21615527050978905
key  1  value[6]  0.25465259711919624
key  2  value[6]  0.3081780004228992
key  3  value[6]  0.39107838594488675
key  4  value[6]  0.5474158342212556
key  5  value[6]  1.0
key  0 value[2]  0.0510486075963949
key  1 value[2]  0.14356443815348044
key  2 value[2]  0.25213556964289846
key  3 value[2]  0.3939282632381201
key  4 value[2]  0.6113773790919804
key  5 value[2]  1.0
Average errors so far np.mean(errList) 0.07491498751394546
Current Error err_H 0.09086624176295696
myFeeder.t  5
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
Output of each sub-model [[0.         0.00701519 0.01272912 ... 0.01662708 0.         0.        ]
 [0.         0.00701519 0.01283096 ... 0.01694378 0.         0.        ]
 [0.         0.00571939 0.01354379 ... 0.01733967

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number o

Start adding new sub-models
Randomly selected indique_sousModle  R_Forest
key  0  value[6]  0.11586412658816056
key  1  value[6]  0.1288294285628787
key  2  value[6]  0.1444116730248319
key  3  value[6]  0.16334850821192484
key  4  value[6]  0.18670485439918758
key  5  value[6]  0.21615527050978905
key  6  value[6]  0.25465259711919624
key  7  value[6]  0.3081780004228992
key  8  value[6]  0.39107838594488675
key  9  value[6]  0.5474158342212556
key  10  value[6]  1.0
key  0 value[2]  0.039306838742498774
key  1 value[2]  0.07646112794962631
key  2 value[2]  0.1157932804570845
key  3 value[2]  0.15956644404871112
key  4 value[2]  0.21045596326717006
key  5 value[2]  0.2682022759660285
key  6 value[2]  0.34123504736392685
key  7 value[2]  0.4299624077906989
key  8 value[2]  0.5462065466048419
key  9 value[2]  0.7043760355728464
key  10 value[2]  1.0
Average errors so far np.mean(errList) 0.06860288155493297
Current Error err_H 0.061026288840658305
myFeeder.t  10
[[0. 0. 0. ... 0. 0. 0.]

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


Start adding new sub-models
Randomly selected indique_sousModle  Lasso


/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


Start adding new sub-models
Randomly selected indique_sousModle  KNN
key  1  value[6]  0.11586412658816056
key  2  value[6]  0.1288294285628787
key  3  value[6]  0.1444116730248319
key  4  value[6]  0.16334850821192484
key  5  value[6]  0.18670485439918758
key  6  value[6]  0.21615527050978905
key  7  value[6]  0.25465259711919624
key  8  value[6]  0.3081780004228992
key  9  value[6]  0.39107838594488675
key  10  value[6]  0.5474158342212556
key  11  value[6]  1.0
key  1 value[2]  0.04028342609023948
key  2 value[2]  0.0769507041032482
key  3 value[2]  0.11732652156883022
key  4 value[2]  0.1638006653208015
key  5 value[2]  0.21542734863188748
key  6 value[2]  0.2786729678652965
key  7 value[2]  0.3521722991452645
key  8 value[2]  0.44280195170686026
key  9 value[2]  0.5552471780392491
key  10 value[2]  0.7171003470835221
key  11 value[2]  1.0
Average errors so far np.mean(errList) 0.06725762117047643
Current Error err_H 0.053805017325911135
myFeeder.t  11
[[0. 0. 0. ... 0. 0. 0.]
 [0.

KeyboardInterrupt: 